In [2]:
import tensorflow as tf
import numpy as np
import xml.etree.ElementTree as ET
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer

/Users/xirong/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
print('Reading corpus and finding features')
xmlcorpus = ET.parse('../Corpus/all.xml')
textes = xmlcorpus.findall('.//text')
corpus = []
for i in textes:
    corpus.append(i.text)
# les étiquettes du document (références)
docs= xmlcorpus.getroot().getchildren()
Y = np.zeros((len(docs), 1))
for i in range(len(docs)):
    doc = docs[i]
    fake = 0
    if doc.get('class') == 'fake':
        fake = 1
    Y[i,0] = fake

Reading corpus and finding features


In [5]:
Y

array([[1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],

In [9]:
vectorizer = CountVectorizer()
transformer = TfidfTransformer()
counts = vectorizer.fit_transform(corpus)
tfidf = transformer.fit_transform(counts)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [5]:
counts.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 3, 0]], dtype=int64)

In [6]:
X = tfidf.toarray()
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.04821643, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.09863567,
        0.        ]])

In [8]:
# définir la taille du batch de donnée d'entraînment
batch_size = 5
dataset_size = X.shape[0]
lenvec = X.shape[1]
# définir les paramètres de réseau de neurones
w1 = tf.Variable(tf.random_normal([lenvec,3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3,1], stddev=1, seed=1))

In [10]:
X.shape

(118, 12070)

In [ ]:
分数据集（训练集，测试集 ）

In [10]:
"""
在shape的一个维度上使用None可以方便使用不大的batch大小。(None: 不定)
"""
x = tf.placeholder(tf.float32, shape=(None, lenvec), name = 'x-input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name = 'y-input')

In [11]:
# 定义神经网络前向传播过程
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

In [12]:
# 定义损失函数来刻画预测值和真实值的差距
# y_代表正确结果，y代表预测结果
# clip_by_value函数可以吧张量中的数值限定在一个范围之中， 这样可以避免一些运算错误（比如log0是无效的）
# tf.clip_by_value(y, 1e-10, 1.0) 就是把y值限定在1e-10与1.0之间
# reduce_mean()对整个矩阵求平均， 用于得到一个batch的平均交叉熵
cross_entropy = -tf.reduce_mean(
    y_*tf.log(tf.clip_by_value(y, 1e-10, 1.0)))
# 定义学习率
learning_rate = 0.001
# 定义反向传播算法来优化神经网络中的参数
algo_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [13]:
# 创建一个会话来运行tf程序
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    # 初始化变量
    sess.run(init_op)
    # 在训练之前神经网络参数的值
    print("训练之前神经网络参数的值")
    print(sess.run(w1))
    print(sess.run(w2))
        # 设定训练的轮数
    steps = 5000
    for i in range(steps):
        # 每次选取batch_size个样本进行训练
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size, dataset_size)

        # 通过选取的样本训练神经网络并更新参数
        sess.run(algo_op, feed_dict = {x: X[start:end], y_:Y[start:end]})
        if i%1000 == 0:
            # 每隔一段时间计算在所有数据上的交叉熵并输出
            total_cross_entropy = sess.run(cross_entropy, feed_dict = {x: X, y_: Y})
            print("After {} training step(s), cross entropy on all data is {}".format(i, total_cross_entropy))
    print("训练后的神经网络参数的值")
    print(sess.run(w1))
    print(sess.run(w2))

训练之前神经网络参数的值
[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]
 [ 0.59282297 -2.1229296  -0.72289723]
 ...
 [ 1.3733915   0.13433133 -1.415918  ]
 [-0.20769614  1.2466617   1.8288933 ]
 [-0.41405147  0.6377421  -0.9574355 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
After 0 training step(s), cross entropy on all data is 5.548383712768555
After 1000 training step(s), cross entropy on all data is 2.9308037757873535
After 2000 training step(s), cross entropy on all data is 1.5655864477157593
After 3000 training step(s), cross entropy on all data is 1.5613031387329102
After 4000 training step(s), cross entropy on all data is 1.5610747337341309
训练后的神经网络参数的值
[[-0.86080456  1.5360957   0.09585378]
 [-2.4427042   0.0992484   0.5912243 ]
 [ 0.5833129  -2.11339    -0.7137513 ]
 ...
 [ 1.3425592   0.16571797 -1.3868986 ]
 [-0.26227173  1.3022805   1.849376  ]
 [-0.41405147  0.6377421  -0.9574355 ]]
[[-0.71392375]
 [ 1.6129307 ]
 [-0.04042413]]
